<a href="https://colab.research.google.com/github/bibekpaudyal1/regressionDataScienceHousePredctionCompleteModel/blob/models/Bibek_paudyal_housing_price_pred_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#mounting the google drive to import the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#checking the contents of the google drive to see the files
import os
directory_path = '/content/drive/MyDrive/'
contents = os.listdir(directory_path)
print(contents)

['How to get started with Drive.pdf', 'Beat my score: 0 in #JumpyTile \n\n http:  play.google.com store apps details?id=com.paxplay.jumpytile t my score:.txt', 'ab.jpg', 'IdeaBreed Website Issues :.gdoc', '45654847_2314904868743610_2772296966028132352_n.jpg', '45685695_463749644149235_4320848141860470784_n.jpg', '45685759_316994612451731_3565131406130544640_n.jpg', '45696401_2018573308165368_2429529829591220224_n.jpg', '45707322_2049762845045984_8733605195126145024_n.jpg', '45710319_306267663543366_5204752561701126144_n.jpg', '45726670_2187608371513577_3053941967609135104_n.jpg', '45775906_277556482895060_4943889755814232064_n.jpg', '45808162_497782837401081_7027150732067340288_n.jpg', '45808205_725047987860383_1213369527165779968_n.jpg', '45829469_182114292695791_3960746418632654848_n.jpg', '45861049_2432408686775029_1092437567531384832_n.jpg', '45915510_2178323715717239_8509815503261270016_n.jpg', '46028792_2015123461908761_7089323201175486464_n.jpg', 'ASVI.jpg', 'Birendra-Tal-Below-

In [4]:
#importing the excel files

import pandas as pd
import numpy as np

file_path = '/content/drive/MyDrive/real_state.xlsx'

real_state_data = pd.read_excel(file_path)
print(real_state_data.columns)




Index(['No', 'X1 transaction date', 'X2 house age',
       'X3 distance to the nearest MRT station',
       'X4 number of convenience stores', 'X5 latitude', 'X6 longitude',
       'Y house price of unit area'],
      dtype='object')


In [5]:
real_state_data.info()
print("---------------")
real_state_data.describe()
print("------------------")
print(real_state_data.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   No                                      414 non-null    int64  
 1   X1 transaction date                     414 non-null    float64
 2   X2 house age                            414 non-null    float64
 3   X3 distance to the nearest MRT station  414 non-null    float64
 4   X4 number of convenience stores         414 non-null    int64  
 5   X5 latitude                             414 non-null    float64
 6   X6 longitude                            414 non-null    float64
 7   Y house price of unit area              414 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 26.0 KB
---------------
------------------
No                                        414
X1 transaction date                        12
X2 house age                

In [6]:
#From the dataset it has already specified that there are no null values but still we will check them

for k in sorted(real_state_data.columns):
  if real_state_data[k].isna().sum()>0:
    print(k,real_state_data[k].isna().sum())
  else:
    print("There are no missing values in the datasets columns")

There are no missing values in the datasets columns
There are no missing values in the datasets columns
There are no missing values in the datasets columns
There are no missing values in the datasets columns
There are no missing values in the datasets columns
There are no missing values in the datasets columns
There are no missing values in the datasets columns
There are no missing values in the datasets columns


In [7]:
#since there are no ny missing values in the dataset we don't need to fill the missing values
#checking for the constant columns in the data

(real_state_data.nunique()==1).sum()

0

In [8]:
#we don't have any constant columns so we don't need to remove that if there were dublicate columns
real_state_data = real_state_data.loc[:,(real_state_data !=real_state_data.iloc[0]).any()]

In [9]:
#checking if there are any dublicated indexes or not
real_state_data.duplicated().sum()
print(real_state_data.dtypes)

No                                          int64
X1 transaction date                       float64
X2 house age                              float64
X3 distance to the nearest MRT station    float64
X4 number of convenience stores             int64
X5 latitude                               float64
X6 longitude                              float64
Y house price of unit area                float64
dtype: object


In [10]:
#Now the process involves the pre-processing of the data

#applying the standardScaler for the numberic data in columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num_cols = real_state_data.select_dtypes(include='number').columns
print(num_cols)
real_state_data[num_cols] = scaler.fit_transform(real_state_data[num_cols])

Index(['No', 'X1 transaction date', 'X2 house age',
       'X3 distance to the nearest MRT station',
       'X4 number of convenience stores', 'X5 latitude', 'X6 longitude',
       'Y house price of unit area'],
      dtype='object')


In [11]:
#pre-processing for the categorical data

cat_cols = real_state_data.select_dtypes(include='object').columns
print(cat_cols)
print(real_state_data.dtypes)

#since we don't have the categorical data we don't have to apply hot-encoding, label encoding and target encoding .

Index([], dtype='object')
No                                        float64
X1 transaction date                       float64
X2 house age                              float64
X3 distance to the nearest MRT station    float64
X4 number of convenience stores           float64
X5 latitude                               float64
X6 longitude                              float64
Y house price of unit area                float64
dtype: object


In [12]:
#cheking the variance and correlation in the columns for effective machine learning model
real_state_data = real_state_data.drop(['No','X1 transaction date'],axis=1)
variances = real_state_data.var()
high_variances = variances[variances>0.5].index
real_state_data = real_state_data[high_variances]


corr = real_state_data.corr()
target_corr = corr['Y house price of unit area']
corr_threshold = 0.7
columns = target_corr[target_corr.abs()>corr_threshold].index



In [13]:
#splitting the dataset into trainin testing and validation data

from sklearn.model_selection import train_test_split
train_val_set,test_val = train_test_split(real_state_data,test_size=0.2,random_state=42)
train_set,val_test = train_test_split(train_val_set,test_size=0.2,random_state=42)

X_train = train_set.drop('Y house price of unit area',axis =1)
X_val = val_test.drop('Y house price of unit area',axis =1)
X_test = test_val.drop('Y house price of unit area',axis =1)

y_train = train_set['Y house price of unit area']
y_val = val_test['Y house price of unit area']
y_test = test_val['Y house price of unit area']


In [14]:
pip install xgBoost

In [15]:

#training our data using the xgboost similar to the random forest
# import xgboost
# from sklearn.model_selection import cross_val_score

# reg = xgboost.XGBRegressor(random_state = 42,
#  early_stopping_rounds = 15,
#  learning_rate=0.1,
#  max_depth = 9,
#  n_estimators=100000,
#  min_child_weight=1,
#  gamma=0,
#  subsample=0.8,
#  colsample_bytree=0.8,
#  n_jobs=-1,
#  verbosity=0)

# reg.fit(X_train, y_train, eval_set = [(X_val, y_val)])


In [16]:
#using the GridSearchCV to test with different parameters to find the best model within a range

import xgboost
from sklearn.model_selection import GridSearchCV

xgb1 = xgboost.XGBRegressor(random_state=42, early_stopping_rounds=15, n_jobs=-1, verbosity=0)

parameters = {
    'nthread': [4],
    'objective': ['reg:linear'],
    'learning_rate': [0.03, 0.05, 0.07],
    'max_depth': [5, 6, 7],
    'min_child_weight': [4],
    'silent': [1],
    'subsample': [0.7],
    'colsample_bytree': [0.7],
    'n_estimators': [500]
}

xgb_grid = GridSearchCV(xgb1, parameters, cv=2, n_jobs=5, verbose=True)

xgb_grid.fit(X_train, y_train, eval_set=[(X_val, y_val)])

print("Best Parameters:", xgb_grid.best_params_)
print("Best Score:", xgb_grid.best_score_)

best_model = xgb_grid.best_estimator_


Fitting 2 folds for each of 9 candidates, totalling 18 fits
[0]	validation_0-rmse:0.94001
[1]	validation_0-rmse:0.91967
[2]	validation_0-rmse:0.90106
[3]	validation_0-rmse:0.88178
[4]	validation_0-rmse:0.86427
[5]	validation_0-rmse:0.84743
[6]	validation_0-rmse:0.83208
[7]	validation_0-rmse:0.81483
[8]	validation_0-rmse:0.79983
[9]	validation_0-rmse:0.78441
[10]	validation_0-rmse:0.77119
[11]	validation_0-rmse:0.75744
[12]	validation_0-rmse:0.74547
[13]	validation_0-rmse:0.73336
[14]	validation_0-rmse:0.72069
[15]	validation_0-rmse:0.70948
[16]	validation_0-rmse:0.69796
[17]	validation_0-rmse:0.68669
[18]	validation_0-rmse:0.67620
[19]	validation_0-rmse:0.66531
[20]	validation_0-rmse:0.65543
[21]	validation_0-rmse:0.64580
[22]	validation_0-rmse:0.63773
[23]	validation_0-rmse:0.63035
[24]	validation_0-rmse:0.62341
[25]	validation_0-rmse:0.61644
[26]	validation_0-rmse:0.60878
[27]	validation_0-rmse:0.60056
[28]	validation_0-rmse:0.59348
[29]	validation_0-rmse:0.58760
[30]	validation_0-rm

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
y_pred = best_model.predict(X_test)

print(f"MAE : {mean_absolute_error(y_test, y_pred)}")
print(f"MSE : {mean_squared_error(y_test, y_pred)}")

MAE : 0.3224049899394488
MSE : 0.20931568856690871


In [18]:
#using other regression model to compare our model and also using the GridSearchCV to test the model using different parameters to see the bese MSE

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [19]:
from sklearn.model_selection import GridSearchCV

# AdaBoost
adaboost_model = AdaBoostRegressor(random_state=42)
adaboost_params = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1],
    'loss': ['linear', 'square', 'exponential'],
}

adaboost_grid = GridSearchCV(adaboost_model, adaboost_params, cv=5, scoring='neg_mean_squared_error')
adaboost_grid.fit(X_train, y_train)
adaboost_best_model = adaboost_grid.best_estimator_
adaboost_predictions = adaboost_best_model.predict(X_val)
adaboost_score = mean_squared_error(y_val, adaboost_predictions)
print("AdaBoost  MSE:", adaboost_score)


AdaBoost  MSE: 0.27790381439884965


In [20]:
# SVM
svm_model = SVR()

svm_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto'],
}

#svm
svm_grid = GridSearchCV(svm_model, svm_params, cv=5, scoring='neg_mean_squared_error')
svm_grid.fit(X_train, y_train)
svm_best_model = svm_grid.best_estimator_
svm_predictions = svm_best_model.predict(X_val)
svm_score = mean_squared_error(y_val, svm_predictions)
print("SVM MSE:", svm_score)

SVM MSE: 0.23831147087909196


In [ ]:
# Random Forest
rf_model = RandomForestRegressor(random_state=42)
rf_params = {
    'n_estimators': [300, 400, 500],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
}
rf_grid = GridSearchCV(rf_model, rf_params, cv=5, scoring='neg_mean_squared_error')
rf_grid.fit(X_train, y_train)
rf_best_model = rf_grid.best_estimator_
rf_predictions = rf_best_model.predict(X_val)
rf_score = mean_squared_error(y_val, rf_predictions)
print("Random Forest  MSE:", rf_score)

In [ ]:
# LightGBM
#comparing all the model and we found out that XGBhas the less mean square error so it's the perfect model for us
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_params = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [300, 400, 500],
    'max_depth': [5, 10, 15],
    'num_leaves': [31, 63, 127],
    'min_child_samples': [20, 40, 60],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5],
}
lgb_grid = GridSearchCV(lgb_model, lgb_params, cv=5, scoring='neg_mean_squared_error')
lgb_grid.fit(X_train, y_train)
lgb_best_model = lgb_grid.best_estimator_
lgb_predictions = lgb_best_model.predict(X_val)
lgb_score = mean_squared_error(y_val, lgb_predictions)
print("LightGBM MSE:", lgb_score)